In [1]:
import numpy as np
import pandas as pd
import respy as rp
from respy.solve import calculate_emax_value_functions

In [2]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
optim_paras, _ = rp.pre_processing.model_processing.process_params_and_options(params, options)

In [3]:
state_space = rp.solve(params, options)

In [4]:
states = pd.DataFrame(state_space.states, columns=["period", "exp_a", "exp_b", "exp_edu", "previous_choice", "type"])
states.describe()

,period,exp_a,exp_b,exp_edu,previous_choice,type
count,317367.000000,317367.000000,317367.000000,317367.000000,317367.000000,317367.0
mean,30.518838,8.734456,8.734456,14.315471,1.492925,0.0
std,7.011614,7.023967,7.023967,3.029777,1.124330,0.0
min,0.000000,0.000000,0.000000,10.000000,0.000000,0.0
25%,26.000000,3.000000,3.000000,12.000000,0.000000,0.0
50%,32.000000,7.000000,7.000000,14.000000,1.000000,0.0
75%,36.000000,13.000000,13.000000,17.000000,3.000000,0.0
max,39.000000,39.000000,39.000000,20.000000,3.000000,0.0


In [5]:
wages = pd.DataFrame(state_space.wages, columns=["wage_a", "wage_b", "wage_edu", "wage_home"])
wages.describe()

,wage_a,wage_b,wage_edu,wage_home
count,317367.000000,317367.000000,317367.0,317367.0
mean,21935.480043,25077.840710,1.0,1.0
std,4018.906860,8726.591695,0.0,0.0
min,14617.869534,9701.152773,1.0,1.0
25%,18882.672848,18324.605526,1.0,1.0
50%,21461.158524,23659.026706,1.0,1.0
75%,24636.885266,30454.833617,1.0,1.0
max,36552.322272,60475.886843,1.0,1.0


In [6]:
nonpecs = pd.DataFrame(state_space.wages, columns=["nonpec_a", "nonpec_b", "nonpec_edu", "nonpec_home"])
nonpecs.describe()

,nonpec_a,nonpec_b,nonpec_edu,nonpec_home
count,317367.000000,317367.000000,317367.0,317367.0
mean,21935.480043,25077.840710,1.0,1.0
std,4018.906860,8726.591695,0.0,0.0
min,14617.869534,9701.152773,1.0,1.0
25%,18882.672848,18324.605526,1.0,1.0
50%,21461.158524,23659.026706,1.0,1.0
75%,24636.885266,30454.833617,1.0,1.0
max,36552.322272,60475.886843,1.0,1.0


In [7]:
periodic_draws = np.dot(state_space.base_draws_sol, optim_paras["shocks_cholesky"])
periodic_draws[..., :2] = np.exp(periodic_draws[..., :2])

In [19]:
def get_characteristics_of_single_state(idx, state_space):
    wage = state_space.wages[idx]
    nonpec = state_space.nonpec[idx]
    is_inadmissible = state_space.is_inadmissible[idx]
    
    period = state_space.states[idx, 0]
    cont_idx = idx - (state_space.states[:, 0] < period).sum()
    
    continuation_values = state_space.get_continuation_values(period)[cont_idx]
    
    periodic_draws = np.dot(state_space.base_draws_sol, optim_paras["shocks_cholesky"])
    periodic_draws[..., :2] = np.exp(periodic_draws[..., :2])
    
    draws = periodic_draws[period]
    
    return wage, nonpec, continuation_values, draws, is_inadmissible

In [20]:
wage, nonpec, continuation_values, draws, is_inadmissible = get_characteristics_of_single_state(1, state_space)

In [23]:
calculate_emax_value_functions(wage, nonpec, continuation_values, draws, optim_paras["delta"], is_inadmissible)

358294.1740142131